# 8. 應用數據庫

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
from pandas import Series, DataFrame

## 8.1 基於 SQLite3 的輕量級訪問

In [3]:
import sqlite3

with sqlite3.connect(":memory:") as con:
    c = con.cursor()
    c.execute('''CREATE TABLE sensors
                 (date text, city text, code text, sensor_id real, temperature real)''')

    for table in c.execute("SELECT name FROM sqlite_master WHERE type = 'table'"):
        print("Table", table[0])

    c.execute("INSERT INTO sensors VALUES ('2016-11-05','Utrecht','Red',42,15.14)")
    c.execute("SELECT * FROM sensors")
    print(c.fetchone())


    con.execute("DROP TABLE sensors")

    print("# of tables", c.execute("SELECT COUNT(*) FROM sqlite_master WHERE type = 'table'").fetchone()[0])

    c.close()

Table sensors
('2016-11-05', 'Utrecht', 'Red', 42.0, 15.14)
# of tables 0


## 8.2 通過 pandas 訪問數據庫

In [4]:
import statsmodels
from pandas.io.sql import read_sql
import sqlite3

with sqlite3.connect(":memory:") as con:
    c = con.cursor()

    data_loader = statsmodels.datasets.sunspots.load_pandas()
    df = data_loader.data
    rows = [tuple(x) for x in df.values]  # 由 DataFrame 中提取資料，準備寫入資料庫

    con.execute("CREATE TABLE sunspots(year, sunactivity)")
    con.executemany("INSERT INTO sunspots(year, sunactivity) VALUES (?, ?)", rows)  # executemany 寫入多筆資料到資料庫
    c.execute("SELECT COUNT(*) FROM sunspots")  # 查看總共有多少筆資料
    print(c.fetchone()) 
    print("Deleted", con.execute("DELETE FROM sunspots where sunactivity > 20").rowcount, "rows")  # 刪除 sunactivity > 20 的資料

    print(read_sql("SELECT * FROM sunspots where year < 1732", con))  # 列出 year < 1732 的資料
    con.execute("DROP TABLE sunspots")  # 刪除 table

    c.close()

(309,)
Deleted 217 rows
      year  sunactivity
0   1700.0          5.0
1   1701.0         11.0
2   1702.0         16.0
3   1707.0         20.0
4   1708.0         10.0
5   1709.0          8.0
6   1710.0          3.0
7   1711.0          0.0
8   1712.0          0.0
9   1713.0          2.0
10  1714.0         11.0
11  1723.0         11.0


## 8.3 SQLAlchemy

In [5]:
import pkgutil as pu
import pydoc
import sqlalchemy as sa

print("sqlalchemy version", sa.__version__)

def clean(astr):
    s = astr
    # remove multiple spaces
    s = ' '.join(s.split())
    s = s.replace('=','')

    return s

def print_desc(prefix, pkg_path):
    for pkg in pu.iter_modules(path=pkg_path):
        name = prefix + "." + pkg[1]

        if pkg[2] == True:
            try:
                docstr = pydoc.plain(pydoc.render_doc(name))
                docstr = clean(docstr)
                start = docstr.find("DESCRIPTION")
                docstr = docstr[start: start + 140]
                print(name, docstr)
            except:
                continue

print_desc("sqlalchemy", sa.__path__)

sqlalchemy version 1.1.9
sqlalchemy.connectors DESCRIPTION # connectors/__init__.py # Copyright (C) 2005-2017 the SQLAlchemy authors and contributors # <see AUTHORS file> # # This module 
sqlalchemy.databases DESCRIPTION Include imports from the sqlalchemy.dialects package for backwards compatibility with pre 0.6 versions. PACKAGE CONTENTS DATA __
sqlalchemy.dialects DESCRIPTION # dialects/__init__.py # Copyright (C) 2005-2017 the SQLAlchemy authors and contributors # <see AUTHORS file> # # This module is
sqlalchemy.engine DESCRIPTION The engine package defines the basic components used to interface DB-API modules with higher-level statement construction, conne
sqlalchemy.event DESCRIPTION # event/__init__.py # Copyright (C) 2005-2017 the SQLAlchemy authors and contributors # <see AUTHORS file> # # This module is pa
sqlalchemy.ext DESCRIPTION # ext/__init__.py # Copyright (C) 2005-2017 the SQLAlchemy authors and contributors # <see AUTHORS file> # # This module is part
sqlalchemy.orm D

### 8.3.1 SQLAlchemy 的安裝與配置

In [6]:
from sqlalchemy import Column, ForeignKey, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
from sqlalchemy import UniqueConstraint
 
Base = declarative_base()  # Base 類別
 
class Station(Base):  # Station 類別
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    name = Column(String(14), nullable=False, unique=True)

    def __repr__(self):
        return "Id=%d name=%s" %(self.id, self.name)

    
class Sensor(Base):  # Sensor 類別
    __tablename__ = 'sensor'
    id = Column(Integer, primary_key=True)
    last = Column(Integer)
    multiplier = Column(Float)
    station_id = Column(Integer, ForeignKey('station.id'))
    station = relationship(Station)

    def __repr__(self):
        return "Id=%d last=%d multiplier=%.1f station_id=%d" %(self.id, self.last, self.multiplier, self.station_id)

    
if __name__ == "__main__":
    print("This script is used by another script. Run python alchemy_query.py")

This script is used by another script. Run python alchemy_query.py


## 8.3.2 通過 SQLAlchemy 填充數據庫

In [7]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
 
from alchemy_entities import Base, Sensor, Station
 
def populate(engine):
    Base.metadata.bind = engine
     
    DBSession = sessionmaker(bind=engine)  # 建立一個 session maker
    session = DBSession()  # 使用 sessionmaker 建立一個 session
     
    de_bilt = Station(name='De Bilt')  # 建立 Station 物件
    session.add(de_bilt)  # 將剛剛建立的 Station 物件加入 session
    session.add(Station(name='Utrecht'))  # 將一個新的 Station 物件加入 session
    session.commit()  # commit 變動
    print("Station", de_bilt)
     
    temp_sensor = Sensor(last=20, multiplier=.1, station=de_bilt)  # 建立 Sensor 物件，其中 station 是一個 foreign 物件
    session.add(temp_sensor)  # 將剛剛建立的 Sensor 物件加入 session
    session.commit()  # commit 變動
    print("Sensor", temp_sensor)

if __name__ == "__main__":
    print("This script is used by another script. Run python alchemy_query.py")

This script is used by another script. Run python alchemy_query.py


## 8.3.2 通過 SQLAlchemy 查詢數據庫

In [8]:
import os
f = 'demo.db'
if os.path.exists(f):
    os.remove(f)

In [9]:
from alchemy_entities import Base, Sensor, Station
from populate_db import populate
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.pool import StaticPool, NullPool
import os
from pandas.io.sql import read_sql


# engine = create_engine('sqlite:///demo.db', poolclass=NullPool)  # 建立 connection
engine = create_engine('sqlite:///demo.db')  # 建立 connection
conn = engine.connect()
Base.metadata.create_all(engine)  # 建立資料庫
populate(engine)  # 在資料庫中 建立資料
Base.metadata.bind = engine
DBSession = sessionmaker()   # 建立一個 session maker
DBSession.bind = engine
session = DBSession()   # 使用 sessionmaker 建立一個 session

station = session.query(Station).first()  # 查詢 Station 資料表中 第一筆資料

print("Query 1", session.query(Station).all())  # 查詢 Station 資料表中 所有資料
print("Query 2", session.query(Sensor).all())  # 查詢 Sensor 資料表中 所有資料
print("Query 3", session.query(Sensor).filter(Sensor.station == station).one())  # 查詢 Sensor 資料表中 符合條件的資料
print(read_sql("SELECT * FROM station", engine.raw_connection()))  # 使用 pandas.read_sql() 從資料庫中讀取資料

try:
    session.close()
    conn.invalidate()    
    conn.close()
#     session = None
    engine.dispose()
    engine = None
    del engine
    os.remove('demo.db')
    print("Deleted demo.db")
except OSError as e:
    print(e)

Station Id=1 name=De Bilt
Sensor Id=1 last=20 multiplier=0.1 station_id=1
Query 1 [Id=1 name=De Bilt, Id=2 name=Utrecht]
Query 2 [Id=1 last=20 multiplier=0.1 station_id=1]
Query 3 Id=1 last=20 multiplier=0.1 station_id=1
   id     name
0   1  De Bilt
1   2  Utrecht
[WinError 32] 程序無法存取檔案，因為檔案正由另一個程序使用。: 'demo.db'


## 8.4 Pony ORM

In [10]:
# from pony.orm import Database, db_session 
# from pandas.io.sql import write_frame
# import statsmodels.api as sm

# db = Database('sqlite', ':memory:')

# with db_session:
#     data_loader = sm.datasets.sunspots.load_pandas()
#     df = data_loader.data
#     write_frame(df, "sunspots", db.get_connection()) 
#     print(db.select("count(*) FROM sunspots"))

## 8.5 Dataset: 懶人數據庫

In [11]:
# import dataset
# from pandas.io.sql import read_sql
# from pandas.io.sql import write_frame
# import statsmodels.api as sm

# db = dataset.connect('sqlite:///:memory:')
# table = db["books"]
# table.insert(dict(title="NumPy Beginner's Guide", author='Ivan Idris'))
# table.insert(dict(title="NumPy Cookbook", author='Ivan Idris'))
# table.insert(dict(title="Learning NumPy", author='Ivan Idris'))
# print(read_sql('SELECT * FROM books', db.executable.raw_connection())) 

# data_loader = sm.datasets.sunspots.load_pandas()
# df = data_loader.data
# write_frame(df, "sunspots", db.executable.raw_connection()) 
# table = db['sunspots']

# for row in table.find(_limit=5):
#    print(row)

# print("Tables", db.tables)

## 8.6 PyMongo 與 MongoDB

In [12]:
from pymongo import MongoClient
import statsmodels as sm
import json
import pandas as pd


client = MongoClient()
db = client.test_database
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test_database')

In [13]:
# data_loader = sm.datasets.sunspots.load_pandas()
# df = data_loader.data
# rows = list(json.loads(df.T.to_json()).values())
# db.sunspots.insert_many(rows)  # 寫入資料庫

# cursor = db['sunspots'].find({})
# df =  pd.DataFrame(list(cursor))
# print(df)

# db.drop_collection('sunspots')

## 8.7 利用 Redis 儲存數據

In [14]:
import redis
import statsmodels.api as sm
import pandas as pd

r = redis.StrictRedis()
r

StrictRedis<ConnectionPool<Connection<host=localhost,port=6379,db=0>>>

In [15]:
# data_loader = sm.datasets.sunspots.load_pandas()
# df = data_loader.data
# data = df.T.to_json()
# r.set('sunspots', data)
# blob = r.get('sunspots')
# print(pd.read_json(blob))

## 8.8 Apache Cassandra

In [16]:
# from cassandra import ConsistencyLevel
# from cassandra.cluster import Cluster
# from cassandra.query import SimpleStatement
# import statsmodels.api as sm

# cluster = Cluster()
# session = cluster.connect()
# session.execute("CREATE KEYSPACE IF NOT EXISTS mykeyspace WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 };")
# session.set_keyspace('mykeyspace')
# session.execute("CREATE TABLE IF NOT EXISTS sunspots (year decimal PRIMARY KEY, sunactivity decimal);")

# query = SimpleStatement(
#     "INSERT INTO sunspots (year, sunactivity) VALUES (%s, %s)",
#     consistency_level=ConsistencyLevel.QUORUM)

# data_loader = sm.datasets.sunspots.load_pandas()
# df = data_loader.data
# rows = [tuple(x) for x in df.values]

# for row in rows:
#     session.execute(query, row)

# print(session.execute("SELECT COUNT(*) FROM sunspots"))

# session.execute('DROP KEYSPACE mykeyspace') 
# cluster.shutdown()